In [21]:
#pip install pdfminer.six
#pip install doc2text
# pip install nltk
#pip install spacy
#python -m spacy download en_core_web_sm

In [6]:
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import io
import nltk
import json
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
from datetime import datetime
import spacy
import pandas as pd
from spacy.matcher import Matcher


def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, caching=True, check_extractable=True):
            resource_manager = PDFResourceManager()
            fake_file_handle = io.StringIO()
            converter = TextConverter(
                                resource_manager, 
                                fake_file_handle, 
                                codec='utf-8', 
                                laparams=LAParams()
                        )
            page_interpreter = PDFPageInterpreter(
                                resource_manager, 
                                converter
                            )
            page_interpreter.process_page(page)
            text = fake_file_handle.getvalue()
            yield text
            converter.close()
            fake_file_handle.close()

def extract_name(resume_text):
    nlp_text = nlp(resume_text)
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}]
    
    matcher.add('NAME', [pattern])
    
    matches = matcher(nlp_text)
    
    for match_id, start, end in matches:
        span = nlp_text[start:end]
        return span.text

def extract_mobile_number(text):
    phone = re.findall(re.compile(r'(?:(?:\+?([1-9]|[0-9][0-9]|[0-9][0-9][0-9])\s*(?:[.-]\s*)?)?(?:\(\s*([2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9])\s*\)|([0-9][1-9]|[0-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9]))\s*(?:[.-]\s*)?)?([2-9]1[02-9]|[2-9][02-9]1|[2-9][02-9]{2})\s*(?:[.-]\s*)?([0-9]{4})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(\d+))?'), text)

    if phone:
        number = ''.join(phone[0])
        if len(number) > 10:
            return '+' + number
        else:
            return number


def extract_email(email):
    email = re.findall("([^@|\s]+@[^@]+\.[^@|\s]+)", email)
    if email:
        try:
            return email[0].split()[0].strip(';')
        except IndexError:
            return None

def extract_skills(resume_text):
    nlp = spacy.load('en_core_web_sm')
    noun_chunks = nlp(resume_text)
    nlp_text = nlp(resume_text)

    tokens = [token.text for token in nlp_text if not token.is_stop]

    data = pd.read_csv("skills.csv") 
    skills = list(data.columns.values)
    
    skillset = []
    for token in tokens:
        if token.lower() in skills:
            skillset.append(token)
 
    for token in noun_chunks:
        token = token.text.lower().strip()
        if token in skills:
            skillset.append(token)
    
    return [i.capitalize() for i in set([i.lower() for i in skillset])]
               
# def extract_dob(dob):
#     dob = re.findall(re.compile(r'([0-9]{2}[/]{1}[0-9]{2}[/]{1}[0-9]{4})'), dob)
#     if dob:
#         try:
#             return datetime.strptime(dob[0], '%d-%m-%Y').strftime('%d-%m-%Y')
#         except ValueError:
#             return None

In [7]:
file_path = 'C:/Users/anude/Downloads/Anudeep Resume.pdf (1).pdf'
text = ''
page = ''
for page in extract_text_from_pdf(file_path):
    text += ' ' + page
    # load pre-trained model
nlp = spacy.load('en_core_web_sm')
# initialize matcher with a vocab
matcher = Matcher(nlp.vocab)

In [8]:
dictionary ={
    "Name" : extract_name(text),
    "Mobile Number" : extract_mobile_number(text),
    "Email" : extract_email(text),
    "Skills" : extract_skills(text)
}

In [9]:
json_object = json.dumps(dictionary, indent = 4)
with open("extracted_details.json", "w") as outfile:
    outfile.write(json_object)
# print(extract_name(text))
# print(extract_mobile_number(text))
# print(extract_email(text))
# print(extract_dob(text))
# print(extract_skills(text))
# print(extract_experience(text))